# Model constructor.

> Create and tune pytorch model.

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#hide
import torch
import torch.nn as nn

from nbdev.showdoc import show_doc
from IPython.display import Markdown, display

In [ ]:
# hide
def print_doc(func_name):
    doc = show_doc(func_name, title_level=4, disp=False)
    display(Markdown(doc))

## ResBlock

In [ ]:
#hide
from model_constructor.model_constructor import ResBlock

In [ ]:
#hide_input
print_doc(ResBlock)

<h4 id="ResBlock" class="doc_header"><code>class</code> <code>ResBlock</code><a href="model_constructor/model_constructor.py#L26" class="source_link" style="float:right">[source]</a></h4>

> <code>ResBlock</code>(**`expansion`**, **`in_channels`**, **`mid_channels`**, **`stride`**=*`1`*, **`conv_layer`**=*`ConvBnAct`*, **`act_fn`**=*`ReLU(inplace=True)`*, **`zero_bn`**=*`True`*, **`bn_1st`**=*`True`*, **`groups`**=*`1`*, **`dw`**=*`False`*, **`div_groups`**=*`None`*, **`pool`**=*`None`*, **`se`**=*`None`*, **`sa`**=*`None`*) :: `Module`

Resnet block

In [ ]:
block = ResBlock(1,64,64)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (act_fn): ReLU(inplace=True)
)

In [ ]:
block = ResBlock(4,64,64,dw=True)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvBnAct(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (act_fn): ReLU(inplace=True)
)

In [ ]:
block = ResBlock(4,64,64,groups=4)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=4, bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvBnAct(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (act_fn): ReLU(inplace=True)
)

In [ ]:
block = ResBlock(2,64,64,act_fn=nn.LeakyReLU(), bn_1st=False)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): ConvBnAct(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (act_fn): LeakyReLU(negative_slope=0.01)
)

In [ ]:
block = ResBlock(2, 32, 64, dw=True)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvBnAct(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (id_conv): Sequential(
    (id_conv): ConvBnAct(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (act_fn): ReLU(inplace=True)
)

In [ ]:
pool = nn.AvgPool2d(2, ceil_mode=True)

In [ ]:
block = ResBlock(2, 32, 64, stride=2, dw=True, pool=pool)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvBnAct(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (id_conv): Sequential(
    (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (id_conv): ConvBnAct(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_

In [ ]:
from model_constructor.layers import SEModule, SimpleSelfAttention

In [ ]:
# collapse_output
block = ResBlock(2, 32, 64, stride=2, dw=True, pool=pool, se=SEModule)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvBnAct(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (se): SEModule(
      (squeeze): AdaptiveAvgPool2d(output_size=1)
      (excitation): Sequential(
        (fc_reduce): Linear(in_features=128, out_features=8, bias=True)
        (se_act): ReLU(inplace=True)
        (fc_expand): Linear(in_features=8, out_features=128,

In [ ]:
# collapse_output
block = ResBlock(2, 32, 64, stride=2, dw=True, pool=pool, se=SEModule, sa=SimpleSelfAttention)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvBnAct(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (se): SEModule(
      (squeeze): AdaptiveAvgPool2d(output_size=1)
      (excitation): Sequential(
        (fc_reduce): Linear(in_features=128, out_features=8, bias=True)
        (se_act): ReLU(inplace=True)
        (fc_expand): Linear(in_features=8, out_features=128,

## Stem, Body, Head

In [ ]:
#hide
from model_constructor.model_constructor import _make_body, _make_head, _make_layer, _make_stem

In [ ]:
#hide_input
print_doc(_make_layer)

<h4 id="_make_layer" class="doc_header"><code>_make_layer</code><a href="model_constructor/model_constructor.py#L88" class="source_link" style="float:right">[source]</a></h4>

> <code>_make_layer</code>(**`expansion`**, **`in_channels`**, **`out_channels`**, **`blocks`**, **`stride`**, **`sa`**)



In [ ]:
#hide_input
print_doc(_make_stem)

<h4 id="_make_stem" class="doc_header"><code>_make_stem</code><a href="model_constructor/model_constructor.py#L75" class="source_link" style="float:right">[source]</a></h4>

> <code>_make_stem</code>()



In [ ]:
#hide_input
print_doc(_make_body)

<h4 id="_make_body" class="doc_header"><code>_make_body</code><a href="model_constructor/model_constructor.py#L99" class="source_link" style="float:right">[source]</a></h4>

> <code>_make_body</code>()



In [ ]:
#hide_input
print_doc(_make_head)

<h4 id="_make_head" class="doc_header"><code>_make_head</code><a href="model_constructor/model_constructor.py#L109" class="source_link" style="float:right">[source]</a></h4>

> <code>_make_head</code>()



## Model Constructor.

In [ ]:
#hide
from model_constructor import ModelConstructor

In [ ]:
#hide_input
print_doc(ModelConstructor)

<h4 id="ModelConstructor" class="doc_header"><code>class</code> <code>ModelConstructor</code><a href="model_constructor/model_constructor.py#L117" class="source_link" style="float:right">[source]</a></h4>

> <code>ModelConstructor</code>(**`name`**=*`'MC'`*, **`in_chans`**=*`3`*, **`num_classes`**=*`1000`*, **`block`**=*`ResBlock`*, **`conv_layer`**=*`ConvBnAct`*, **`block_sizes`**=*`[64, 128, 256, 512]`*, **`layers`**=*`[2, 2, 2, 2]`*, **`norm`**=*`BatchNorm2d`*, **`act_fn`**=*`ReLU(inplace=True)`*, **`pool`**=*`AvgPool2d(kernel_size=2, stride=2, padding=0)`*, **`expansion`**=*`1`*, **`groups`**=*`1`*, **`dw`**=*`False`*, **`div_groups`**=*`None`*, **`sa`**:`Union`\[`bool`, `int`, `typing.Callable`\]=*`False`*, **`se`**:`Union`\[`bool`, `int`, `typing.Callable`\]=*`False`*, **`se_module`**=*`None`*, **`se_reduction`**=*`None`*, **`bn_1st`**=*`True`*, **`zero_bn`**=*`True`*, **`stem_stride_on`**=*`0`*, **`stem_sizes`**=*`[32, 32, 64]`*, **`stem_pool`**=*`MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)`*, **`stem_bn_end`**=*`False`*, **`_init_cnn`**=*`init_cnn`*, **`_make_stem`**=*`_make_stem`*, **`_make_layer`**=*`_make_layer`*, **`_make_body`**=*`_make_body`*, **`_make_head`**=*`_make_head`*)

Model constructor. As default - xresnet18

In [ ]:
mc  = ModelConstructor()
mc

MC constructor
  in_chans: 3, num_classes: 1000
  expansion: 1, groups: 1, dw: False, div_groups: None
  sa: False, se: False
  stem sizes: [3, 32, 32, 64], stride on 0
  body sizes [64, 128, 256, 512]
  layers: [2, 2, 2, 2]

In [ ]:
mc._block_sizes

[64, 128, 256, 512]

In [ ]:
mc.block_sizes

[64, 64, 128, 256, 512]

In [ ]:
mc._block_sizes = [128, 256, 512, 1024]
mc

MC constructor
  in_chans: 3, num_classes: 1000
  expansion: 1, groups: 1, dw: False, div_groups: None
  sa: False, se: False
  stem sizes: [3, 32, 32, 64], stride on 0
  body sizes [128, 256, 512, 1024]
  layers: [2, 2, 2, 2]

In [ ]:
mc.block_sizes

[64, 128, 256, 512, 1024]

In [ ]:
#hide
model = mc()

In [ ]:
#collapse_output
mc.stem

Sequential(
  (conv_0): ConvBnAct(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_1): ConvBnAct(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_2): ConvBnAct(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [ ]:
#collapse_output
mc.stem_stride_on = 1
mc.stem

Sequential(
  (conv_0): ConvBnAct(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_1): ConvBnAct(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_2): ConvBnAct(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [ ]:
mc.bn_1st = False

In [ ]:
mc.act_fn =nn.LeakyReLU(inplace=True)

In [ ]:
mc.sa = SimpleSelfAttention
mc.se = SEModule

In [ ]:
# collapse_output
mc.body.l_0

Sequential(
  (bl_0): ResBlock(
    (convs): Sequential(
      (conv_0): ConvBnAct(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_1): ConvBnAct(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SEModule(
        (squeeze): AdaptiveAvgPool2d(output_size=1)
        (excitation): Sequential(
          (fc_reduce): Linear(in_features=64, out_features=4, bias=True)
          (se_act): ReLU(inplace=True)
          (fc_expand): Linear(in_features=4, out_features=64, bias=True)
          (se_gate): Sigmoid()
        )
      )
    )
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (bl_1): ResBlock(
   

model_constructor
by ayasyrev